## 2e. Calculate closed-system degassing paths for different volatile systems.

## Setting things up
First we need to import a few Python packages (including VolFe) and set the precision of the calculations. You need to install VolFe once on your machine, if you haven't yet, uncomment the line below (remove the #).

In [ ]:
# Install VolFe on your machine. Remove the # from line below to do this (don't remove the # from this line!).
# pip install VolFe

In [8]:
# import python packages
import pandas as pd
import gmpy2 as gp
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import VolFe as vf

# set calculation precision
gp.get_context().precision=50

These are some examples where the volatiles aren't C, H, and S. These types of calculations can be done for open- and closed-system, re- and degassing paths as described in Examples 2a-d, but we'll just show them for closed-system degassing paths where the inputted composition represents the bulk composition of the system.

For the volatile-free melt composition, we'll use Sari15-04-33 from Brounce et al. (2014) at 1200 'C as before, but we will specify different volatile concentrations. 
First, we'll model CHOAr degassing - i.e., carbon, hydrogen, and argon are the volatiles of interest (no sulfur).
In this case the initial volatile content is 500 ppm CO2, 2 wt%, and 100 ppm Ar.
The Ar amount is inputted for the "X" species.

In [2]:
# Define the melt composition, fO2 estimate, and T as a dictionary.
my_analysis = {'Sample':'Sari15-04-33',
           'T_C': 1200., # Temperature in 'C
           'SiO2': 47.89, # wt%
           'TiO2': 0.75, # wt%
           'Al2O3': 16.74, # wt%
           'FeOT': 9.43, # wt%
           'MnO': 0.18, # wt%
           'MgO': 5.92, # wt%
           'CaO': 11.58, # wt%
           'Na2O': 2.14, # wt%
           'K2O': 0.63, # wt%
           'P2O5': 0.17, # wt%
           'H2O': 2., # wt%
           'CO2ppm': 500., # ppm
           'STppm': 0., # ppm
           'Xppm': 100., # ppm <<< treating this as Ar
           'Fe3FeT': 0.195}

# Turn the dictionary into a pandas dataframe, setting the index to 0.
my_analysis = pd.DataFrame(my_analysis, index=[0])

We'll use the default options, but the important options for noble gas modelling are:

**species X** where the default is Ar (*Ar*)

**species X solubility** where the default is for Ar in basalt (*Ar_Basalt_HughesIP*)

Both these defaults are fine for this example, but you can change them to Ar in rhyolite (*Ar_Rhyolite_HughesIP*) or Ne (*Ne*) for basalt (*Ne_Basalt_HughesIP*) or rhyolite (*Ne_Rhyolite_HughesIP*).

In [3]:
# print default options in VolFe
print(vf.default_models)

                option
type                  
insolubles         yes
H2S_m              yes
species X           Ar
Hspeciation       none
fO2           Kress91A
...                ...
bulk_O           exc_S
error              0.1
print status        no
output csv         yes
setup               no

[63 rows x 1 columns]


But I'll just highlight the specific options that are important to the degassing calculation and what their default values are:

**bulk_composition = yes** This means the composition (including volatiles and fO2 estimate) represent the bulk composition of the system - there is only melt present and all volatiles are dissolved in the melt.

**starting_P = bulk** This means the calculation will start at Pvsat for the composition given.

**gasssing_style = closed** This means the melt and vapor remain in chemical equilibrium throughout and the bulk composition is constant during the calculation

**gassing_direction = degas** This means pressure will decrease during the calculations and therefore the melt will degas.

The default pressure step is 10 bars, which works most of the time but can be a little slow.
Increasing the step size will speed up the calculation but if the step size is too big it will struggle to solve because it uses the previous step for its initial guesses.

In [4]:
degas1 = vf.calc_gassing(my_analysis)

We could run the same calculation from a csv file if we preferred.
We'll use the examples_marianas csv in files and use all the default options again. 
The data in this file are from Brounce et al. (2014, 2016) and Kelley & Cottrell (2012).
Sari15-04-33 is row 48 in that file. 
Uncomment the lines to use them.

In [ ]:
# my_analyses = pd.read_csv("../files/example_marianas.csv") 

# vf.calc_gassing(my_analyses,row=48)

Note that we got a warning that the melt was sulfide-saturated during the degassing path. 
Under those conditions, the results for the melt and vapor are metastable with respect to saturation of sulfide.

We can run the same calculation, but instead limit the sulfur content in the melt to sulfide or anhydrite saturation, if it reaches the limit. To do that we change the "sulfur_saturation" option from "no" (sulfur phases cannot saturate) to "yes" (sulfur phases can saturate if the sulfur content is high enough).

In [5]:
# choose the options I want - everything else will use the default options
my_models = [['sulfur_saturation','yes']]

# turn to dataframe with correct column headers and indexes    
my_models = vf.make_df_and_add_model_defaults(my_models)

And then we run the degassing calculation with those model options

In [6]:
degas2 = vf.calc_gassing(my_analysis, models=my_models)

In [21]:
print(degas1["CO2T-eq_ppmw"])

0    1486.9999999999945
0    1477.4084826190519
0    1470.3617301798768
0    1463.3230863765984
0    1456.2922612122384
            ...        
0    1.9974925955885467
0    1.8708223005878519
0    1.7470065324478625
0    1.6260134811624454
0    1.5078145265430241
Name: CO2T-eq_ppmw, Length: 371, dtype: object


And below we can plot the two results to see the difference.
The blue curves are the first calculation (not allowing sulfur to saturate), whilst the red curves are the second calculation (sulfur can saturate as sulfide or anhydrite if the concentration in the melt is high enough).
It has no effect on the H2O or CO2 contents of the melt, but the sulfur content is much less at the start of degassing because it is in a sulfide phase instead!

In [19]:
fig = make_subplots(rows=1, cols=3, shared_yaxes = True, shared_xaxes = False,vertical_spacing=0.08, horizontal_spacing=0.05)

lw = 3
syms = 10
lc1 = "steelblue"
lc2 = "firebrick"

data1 = degas1
data2 = degas2

r = 1
# P vs. CO2
c = 1
fig.add_trace(go.Scatter(mode = "lines", x=data1['CO2T-eq_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['CO2T-eq_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "CO<sub>2</sub>-eq (ppm)", row = r, col = c)
fig.update_yaxes(title = "<i>P</i> (bars)", row = r, col = c)

# P vs. H2O
c = 2
#fig.add_trace(go.Scatter(mode = "lines", x=data1['H2OT-eq_wtpc'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['H2OT-eq_wtpc'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "H<sub>2</sub>O-eq (wt%)", row = r, col = c)

# P vs. S
c = 3
#fig.add_trace(go.Scatter(mode = "lines", x=data1['ST_ppmw'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['ST_ppmw'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "S<sub>T</sub> (ppm)", row = r, col = c)

c = 1
#fig.add_trace(go.Scatter(mode = "lines", x=data1['fO2_DFMQ'], y=data1['P_bar'], line_color = lc1, line_width = lw,line_dash = "solid"), row = r, col = c)
#fig.add_trace(go.Scatter(mode = "lines", x=data2['fO2_DFMQ'], y=data2['P_bar'], line_color = lc2, line_width = lw, line_dash = "solid"), row = r, col = c)
fig.update_xaxes(title = "DFMQ", row = r, col = c)

fig.update_layout(height=400, width=1000, plot_bgcolor='rgb(255,255,255)' , showlegend = False)

fig.update_xaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))
fig.update_yaxes(showline=True, linewidth=1, linecolor='black', mirror=True, ticks="inside", ticklen=5, title_standoff = 0, tickcolor="black",title_font=dict(size=15, family='Helvetica', color='black'), tickfont=dict(family='Helvetica', color='black', size=12))

fig.update_layout(font_family="Helvetica",font_color="black")

fig.show()

TypeError: Object of type mpfr is not JSON serializable